In [1]:
import pandas as pd
filepath = '/Users/danielseymour/Developer/EC334-Summative/raw_data/dynamism_data/bds2022_gr_vcn4.csv'
df = pd.read_csv(filepath, encoding='latin1')


In [6]:
df.head()

,year,fempgr_gr,vcnaics4,firms,estabs,emp,denom,estabs_entry,estabs_entry_rate,estabs_exit,...,job_destruction_deaths,job_destruction_continuers,job_destruction_rate_deaths,job_destruction_rate,net_job_creation,net_job_creation_rate,reallocation_rate,firmdeath_firms,firmdeath_estabs,firmdeath_emp
0,1978,a) -2,1131,0,0,0,27,0,0.000,30,...,54,0,200.000,200.000,-54,-200.000,0.000,22,22,37
1,1978,a) -2,1132,0,0,0,21,0,0.000,16,...,41,0,200.000,200.000,-41,-200.000,0.000,10,10,25
2,1978,a) -2,1133,0,0,0,3475,0,0.000,2028,...,6949,0,200.000,200.000,-6949,-200.000,0.000,1521,1524,5347
3,1978,a) -2,1141,0,0,0,610,0,0.000,359,...,1219,0,200.000,200.000,-1219,-200.000,0.000,272,272,799
4,1978,a) -2,1142,0,0,0,36,0,0.000,31,...,71,0,200.000,200.000,-71,-200.000,0.000,25,25,64


In [3]:
# Check for NaN values in the column
print("Number of NaN values in fempgr_gr:", df['fempgr_gr'].isna().sum())
print("Total rows:", len(df))

# Check data type
print("Data type of fempgr_gr:", df['fempgr_gr'].dtype)

# Look at first few rows
print("\nFirst 10 values of fempgr_gr:")
print(df['fempgr_gr'].head(10))

# Check if the column has any non-NaN values at all
print("\nAny non-NaN values?", df['fempgr_gr'].notna().any())

Number of NaN values in fempgr_gr: 0
Total rows: 116640
Data type of fempgr_gr: object

First 10 values of fempgr_gr:
0    a) -2
1    a) -2
2    a) -2
3    a) -2
4    a) -2
5    a) -2
6    a) -2
7    a) -2
8    a) -2
9    a) -2
Name: fempgr_gr, dtype: object

Any non-NaN values? True


In [7]:
df.columns

Index(['year', 'fempgr_gr', 'vcnaics4', 'firms', 'estabs', 'emp', 'denom',
       'estabs_entry', 'estabs_entry_rate', 'estabs_exit', 'estabs_exit_rate',
       'job_creation', 'job_creation_births', 'job_creation_continuers',
       'job_creation_rate_births', 'job_creation_rate', 'job_destruction',
       'job_destruction_deaths', 'job_destruction_continuers',
       'job_destruction_rate_deaths', 'job_destruction_rate',
       'net_job_creation', 'net_job_creation_rate', 'reallocation_rate',
       'firmdeath_firms', 'firmdeath_estabs', 'firmdeath_emp'],
      dtype='object')

In [8]:
# First, convert emp to numeric (it's currently a string)
df['emp'] = pd.to_numeric(df['emp'], errors='coerce')

# Create binary indicator for high-growth firms (growth >= 0.8)
df['is_high_growth'] = df['fempgr_gr'].isin(['h) [0.8 to 2)', 'i) 2']).astype(int)

# Create employment in high-growth firms
df['emp_high_growth'] = df['emp'] * df['is_high_growth']

# Now create your aggregated dataframe using vcnaics4
hgfs_by_industry = df.groupby(['year', 'vcnaics4']).agg({
    'emp_high_growth': 'sum',
    'emp': 'sum'
}).reset_index()

# Calculate the percentage of high-growth employment
hgfs_by_industry['pct_high_growth_emp'] = (hgfs_by_industry['emp_high_growth'] / hgfs_by_industry['emp']) * 100

# Keep only the columns you want
hgfs_by_industry = hgfs_by_industry[['year', 'vcnaics4', 'pct_high_growth_emp']]

# Sort by year and industry for better readability
hgfs_by_industry = hgfs_by_industry.sort_values(['year', 'vcnaics4']).reset_index(drop=True)

# Display the first few rows
print("High-growth employment percentage by year and industry (4-digit NAICS):")
print(hgfs_by_industry.head(10))

# Check the shape
print(f"\nDataframe shape: {hgfs_by_industry.shape}")

# Save the dataframe to CSV
output_path = '/Users/danielseymour/Developer/EC334-Summative/processed_data/hgfs_by_industry_naics4.csv'
hgfs_by_industry.to_csv(output_path, index=False)
print(f"\nCSV file saved to: {output_path}")
print(f"File contains {len(hgfs_by_industry)} rows")

High-growth employment percentage by year and industry (4-digit NAICS):
   year  vcnaics4  pct_high_growth_emp
0  1978      1131             5.738507
1  1978      1132             8.400927
2  1978      1133            18.464760
3  1978      1141            33.797444
4  1978      1142            45.389049
5  1978      1151            19.125845
6  1978      1152            15.070757
7  1978      1153            36.475145
8  1978      2111             7.053673
9  1978      2121             8.030821

Dataframe shape: (12960, 3)

CSV file saved to: /Users/danielseymour/Developer/EC334-Summative/processed_data/hgfs_by_industry_naics4.csv
File contains 12960 rows
